In [49]:
# Import libraries to build Word2Vec model, and load Newsgroups data
import os
import numpy as np
import sys
import re
from gensim.models import Word2Vec
from gensim.models.phrases import Phraser, Phrases
import pandas as pd

In [88]:
#Import data
df = pd.read_csv("train.csv")
data = df["text"]

#Remove Special Characters and append to 'corpus'
failed_files = []
corpus = []
for i, text in enumerate(data):
    try:
        text_mod = re.sub("[^a-zA-Z]", " ", text)
        text_mod = text_mod.lower()
        text_mod = text_mod.split()

        filt = [w for w in text_mod if len(w) > 1]
        corpus.append(filt)

    except TypeError:
        failed_files.append(i)

print('Der er ',len(data),' artikler i datafilen')
print('Der er ',len(corpus),' artikler i corpus')
print('Forskellen er', len(data)-len(corpus), 'artikler' )

y = np.delete(np.array(df['label']), failed_files)


Der er  20800  artikler i datafilen
Der er  20761  artikler i corpus
Forskellen er 39 artikler


In [96]:
#Import data
df = pd.read_csv("train.csv")
text1 = df["text"]
df2 = pd.read_csv("Fake.csv")
text2 = df2['text']
df3 = pd.read_csv("True.csv")
text3 = df3['text']
data = pd.concat([text1, text2, text3], ignore_index =True)

#Remove Special Characters and append to 'corpus'
failed_files = []
corpus = []
for i, text in enumerate(data):
    try:
        text_mod = re.sub("[^a-zA-Z]", " ", text)
        text_mod = text_mod.lower()
        text_mod = text_mod.split()

        filt = [w for w in text_mod if len(w) > 1]
        corpus.append(filt)
    except TypeError:
        failed_files.append(i)

print('Der er ',len(data),' artikler i datafilen')
print('Der er ',len(corpus),' artikler i corpus')
print('Forskellen er', len(data)-len(corpus), 'artikler' )

y = np.delete(y, failed_files)



KeyboardInterrupt: 

In [89]:
#train model
model = Word2Vec(corpus, min_count=0)

#summarize the loaded model
print(model)

#summarize vocabulary into list
words = list(model.wv.index_to_key)
# print(words)

#access vector for one word
# print(model.wv['trump'])
# print(model.wv.most_similar('fish', topn=10))

#save model
model.save('model.selftrained')

#load model
new_model = Word2Vec.load('model.selftrained')
print(new_model)

Word2Vec(vocab=148095, vector_size=100, alpha=0.025)
Word2Vec(vocab=148095, vector_size=100, alpha=0.025)


In [64]:
word_to_test = 'it'
print(model.wv.most_similar(word_to_test,topn=10))


[('that', 0.7316847443580627), ('he', 0.7073552012443542), ('this', 0.6886634230613708), ('but', 0.6861879229545593), ('something', 0.680363655090332), ('just', 0.6282867789268494), ('she', 0.6176166534423828), ('what', 0.6099803447723389), ('really', 0.6080006957054138), ('simply', 0.6035425662994385)]


In [90]:
vector_length = 100
art_matrix = np.zeros((len(corpus), vector_length))

for i, text in enumerate(corpus):
    vect = [model.wv[w] for w in text]
    x = np.mean(vect, axis=0)
    art_matrix[i] = x

# print(art_matrix)

/Users/beheard/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/beheard/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [93]:
x = np.argwhere(np.isnan(art_matrix))
print(x)
row = 0
for i in x[:, 0]:
    if i > row:
        # print(corpus[i])
        row = i

[[   82     0]
 [   82     1]
 [   82     2]
 ...
 [20519    97]
 [20519    98]
 [20519    99]]


In [94]:
from sklearn.model_selection import train_test_split

# Datasæt 1
X_train, X_test, y_train, y_test = train_test_split(art_matrix, y, random_state=0, shuffle=True)
print("Training articles: {}\nTest articles: {}".format(np.size(y_train), np.size(y_test)))

Training articles: 15570
Test articles: 5191


In [97]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(max_iter=1000).fit(X_train, y_train)
print("Dataset training test score: {}".format(log_reg.score(X_train, y_train)))
print("Dataset test score: {}".format(log_reg.score(X_test, y_test)))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').